In [1]:
from nfl import NFL
import pandas as pd
import numpy as np

nfl = NFL().load()

In [2]:
# include this line to "go back in time" by removing
# scores at the end of the season. Otherwise, comment
# out to consider all scores
nfl.clear([17, 18])

In [3]:
# Display current standings
z = nfl('NFC').standings
z

div overall                    division                conference  \
                    win loss tie       pct      win loss tie   pct        win   
team                                                                            
PHI    NFC-East      11    4   0  0.733333        4    1   0   0.8          7   
DAL    NFC-East      10    5   0  0.666667        4    1   0   0.8          7   
NYG    NFC-East       5   10   0  0.333333        2    3   0   0.4          4   
WAS    NFC-East       4   11   0  0.266667        0    5   0   0.0          2   
DET   NFC-North      11    4   0  0.733333        3    2   0   0.6          7   
MIN   NFC-North       7    8   0  0.466667        2    2   0   0.5          6   
GB    NFC-North       7    8   0  0.466667        2    2   0   0.5          5   
CHI   NFC-North       6    9   0       0.4        2    3   0   0.4          5   
TB    NFC-South       8    7   0  0.533333        3    1   0  0.75          6   
ATL   NFC-South       7    8   0  0.466667        3    2   0   0.6          4   
NO    NFC-South       7    8   0  0.466667        2    2   0   0.5          4   
CAR   NFC-South       2   13   0  0.133333        1    4   0   0.2          1   
SF     NFC-West      11    4   0  0.733333        5    0   0   1.0          9   
STL    NFC-West       8    7   0  0.533333        4    1   0   0.8          6   
SEA    NFC-West       8    7   0  0.533333        1    4   0   0.2          6   
ARI    NFC-West       3   12   0       0.2        0    5   0   0.0          2   

                         
     loss tie       pct  
team                     
PHI     3   0       0.7  
DAL     3   0       0.7  
NYG     6   0       0.4  
WAS     8   0       0.2  
DET     3   0       0.7  
MIN     4   0       0.6  
GB      5   0       0.5  
CHI     5   0       0.5  
TB      4   0       0.6  
ATL     6   0       0.4  
NO      6   0       0.4  
CAR    10   0  0.090909  
SF      1   0       0.9  
STL     4   0       0.6  
SEA     5   0  0.545455  
ARI     8   0       0.2

In [4]:
# 7 teams make the playoffs (4 division champs, 3 wilcards)
# so find the 7th-place team currently and get their number of wins
t = nfl.tiebreaks(z.index).index[6]
min_wins = z.loc[t,('overall','win')]
min_wins

# Roughly estimate which teams have secured wildcard berths
# These are teams that will still have superior records even if
# they lose all remaining games and other teams in contention win theirs
z[z[('overall','win')]>min_wins+2]

div overall                    division               conference  \
                    win loss tie       pct      win loss tie  pct        win   
team                                                                           
PHI    NFC-East      11    4   0  0.733333        4    1   0  0.8          7   
DET   NFC-North      11    4   0  0.733333        3    2   0  0.6          7   
SF     NFC-West      11    4   0  0.733333        5    0   0  1.0          9   

                    
     loss tie  pct  
team                
PHI     3   0  0.7  
DET     3   0  0.7  
SF      1   0  0.9

In [5]:
# Playoff standings, if "season ended this week"
p = nfl('NFC').playoffs()
z[z.index.isin(p.index)].reindex(p.index).assign(seed=p)

div overall                    division                conference  \
                   win loss tie       pct      win loss tie   pct        win   
SF    NFC-West      11    4   0  0.733333        5    0   0   1.0          9   
PHI   NFC-East      11    4   0  0.733333        4    1   0   0.8          7   
DET  NFC-North      11    4   0  0.733333        3    2   0   0.6          7   
TB   NFC-South       8    7   0  0.533333        3    1   0  0.75          6   
DAL   NFC-East      10    5   0  0.666667        4    1   0   0.8          7   
STL   NFC-West       8    7   0  0.533333        4    1   0   0.8          6   
SEA   NFC-West       8    7   0  0.533333        1    4   0   0.2          6   

                            seed  
    loss tie       pct            
SF     1   0       0.9      West  
PHI    3   0       0.7      East  
DET    3   0       0.7     North  
TB     4   0       0.6     South  
DAL    3   0       0.7  Wildcard  
STL    4   0       0.6  Wildcard  
SEA    5   0  0.545455  Wildcard

In [6]:
# Find "on the edge" teams that (roughly estimated)
# have not been statistically eliminated, and report
# in wildcard tiebreaker order
min_win = z.loc[p.index[-1], ('overall','win')]
wc = z[(z[('overall','win')]>=min_win-2) & (z.index.isin(p.index) == False)]
t = nfl.tiebreaks(wc.index)
wc.reindex(t.index)

div overall                    division               conference  \
                   win loss tie       pct      win loss tie  pct        win   
MIN  NFC-North       7    8   0  0.466667        2    2   0  0.5          6   
ATL  NFC-South       7    8   0  0.466667        3    2   0  0.6          4   
NO   NFC-South       7    8   0  0.466667        2    2   0  0.5          4   
GB   NFC-North       7    8   0  0.466667        2    2   0  0.5          5   
CHI  NFC-North       6    9   0       0.4        2    3   0  0.4          5   

                   
    loss tie  pct  
MIN    4   0  0.6  
ATL    6   0  0.4  
NO     6   0  0.4  
GB     5   0  0.5  
CHI    5   0  0.5

In [7]:
# The tiebreaks function eliminates teams one at a time until
# only one remains; the last team is the first eliminated
# and the first team is the final team remaining.
# You can see how each wildcard team compares to (is eliminated by)
# the one(s) just above it and what rule eliminated it.
t

# Use the tiebreakers function to compare groups of teams and
# see how each stage of elimination plays out. Tiebreakers are
# limited since the season isn't complete at this point.

MIN                              winner
ATL                        head-to-head
NO     division-tiebreaker:head-to-head
GB     division-tiebreaker:head-to-head
CHI                             overall
Name: eliminated-by, dtype: object